/home/amir/Codes/NN-dynamic-scaling already in Python path
Cloning module 
Unsupported module type: <class 'src.models.mlp.MLP'>
Cloning module layers
Unsupported module type: <class 'torch.nn.modules.container.ModuleDict'>
Cloning module layers.linear_0
Cloning Linear module: 10→10, 64→128, in expansion: 1, out expansion: 2
Cloning module layers.act_0
Cloning module layers.linear_1
Cloning Linear module: 64→128, 32→64, in expansion: 2, out expansion: 2
Cloning module layers.act_1
Cloning module layers.out
Cloning Linear module: 32→64, 2→2, in expansion: 2, out expansion: 1
All activations match after cloning up to tolerance 0.001
Replaced Flatten with CloneAwareFlatten at layers.flatten
Cloning module 
Unsupported module type: <class 'src.models.cnn.CNN'>
Cloning module layers
Unsupported module type: <class 'torch.nn.modules.container.ModuleDict'>
Cloning module layers.conv_0
Cloning Conv2d module: 3→3, 64→128, in expansion: 1, out expansion: 2
Cloning module layers.norm_0
Cloning mo